# Date Parsing & Extraction

The purpose of this note book is to look at extracting dates from the three Date specific fields in CUAD.

In [157]:
#Import required packages
from DownloadCUAD import DownloadCUAD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import spacy
from spacy import displacy
import en_core_web_sm
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from num2words import num2words
import dateparser

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RichardWarburton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
#Download CUAD Data to local drive
DC = DownloadCUAD()
DC.setup()

CUAD data downloading...
CUAD data downloaded & extracted!


In [372]:
#Concatenate string where there are multiple annotations.
agr_contracts,agr_combined_annotations = DC.extract_clause('Agreement Date',False)
exp_contracts,exp_combined_annotations = DC.extract_clause('Expiration Date',False)
eff_contracts,eff_combined_annotations = DC.extract_clause('Effective Date',False)

In [4]:
#Define Caluse of Interest for investigation
clause_of_interest = 'Agreement Date'

## Section 2: Extract dates from 'Agreement Date' annotations

In [83]:
#Define stopwords list
stop_words = stopwords.words('english')

In [84]:
def ordinalize_num(txt):
    for token in txt.split(sep=' '):
        try:
            if int(token) <= 31:
                ord_val = num2words(token, lang="en", to="ordinal_num")
                txt = txt.replace(token,ord_val)
        except:
            continue
    return txt

In [325]:
date_feature_list = []
not_mapped = []

parsed_annotation = []

nlp = en_core_web_sm.load()

for annotation in agr_combined_annotations:

    annotation_list = []  
    #Clean annotaiton (typos etc)
    
    annotation = annotation.replace('  ',' ')
    annotation = annotation.replace('t h','th')
    annotation = annotation.replace('s t','st')
    annotation = re.sub('day|Day|DAY','',annotation)
    annotation = annotation.replace("'","") 
        
    annotation = ' '.join([word for word in annotation.split(sep=' ') if word not in stop_words])   
    
    annotation = ordinalize_num(annotation)
    
    doc = nlp(repr(annotation))
   
    #If no date parsed
    if annotation_list == []:
        annotation = re.sub('(\d{2})([^\S]{1})(\d{2})',r'\1\3',annotation)
        annotation = annotation.replace('  ',' ')
        doc = nlp(repr(annotation))
        for X in doc.ents:
            if X.label_ in ['DATE']:
                annotation_list.append(X.text)
            else:
                annotation_list.append(annotation)
                not_mapped.append(annotation)
    
    date_feature_list.append(' '.join(annotation_list))

In [363]:
import datetime
i=0

new_features = []

for annotation in agr_combined_annotations:
    new_annotation = re.sub('day|Day|DAY','',annotation)
    new_annotation = re.sub('of|Of|OF','',new_annotation)
    new_annotation = re.sub('[^\S]{2,}',' ',new_annotation)
    new_annotation = new_annotation.replace('t h','th')
    new_annotation = new_annotation.replace('s t','st')
    #new_annotation = ordinalize_num(new_annotation)
    
    parsed_val = dateparser.parse(new_annotation)
    
    #print('Orriginal:',annotation)
    #print('Processed:',new_annotation)
    #print('Parsed:',parsed_val)
    #print('-----')
    
    if parsed_val != None:
        i+=1
        new_features.append(parsed_val)
    else:
        new_features.append(np.datetime64('NaT'))
        

In [369]:
print('Date times extracted from {} out of {} annotations ({:.2%})'.format(*(i,len(new_features),i/len(new_features))))

Date times extracted from 422 out of 476 annotations (88.66%)


In [373]:
date_dict = defaultdict(list)

for ii in range(len(agr_contracts)):
    date_dict[agr_contracts[ii]].append(new_features[ii])

AttributeError: 'list' object has no attribute 'totimestamp'